In [ ]:
import pandas as pd
import random
import numpy as np
import cv2 as cv
import glob
from math import ceil, floor
from matplotlib import pyplot as plt
import os




In [ ]:
lst1 = list(range(100))
lst2 =list(np.random.randint(low = 1,high=3000,size=100))
df = pd.DataFrame(
    {'No': lst1,
     'Bw': lst2,
    })

In [ ]:
df.to_csv('histogram.csv',index=False)

In [ ]:
histogram= df.to_dict('r')

In [ ]:
histogram

In [ ]:
from miditime.miditime import MIDITime
mymidi = MIDITime(300, 'histilist.mid', 70, 3, 3)

In [ ]:
my_data_timed_b = [{'beat': mymidi.beat(d['No']), 'freq': d['Bw']} for d in histogram]

In [ ]:
start_timeb = my_data_timed_b[0]['beat']

In [ ]:
def mag_to_pitch_tunedb(freq):
    scale_pctb = mymidi.linear_scale_pct(min([x['Bw'] for x in histogram]), 3000, freq)


    c_major = ["C", "C#", "Db", "D", "D#", "Eb", "E", "F", "F#", "Gb", "G", "G#", "Ab", "A", "A#", "Bb", "B"]


    noteb = mymidi.scale_to_note_classic(scale_pctb, c_major)



    midi_pitchb = mymidi.note_to_midi_pitch(noteb)

    
    return midi_pitchb



In [ ]:
note_listb = []

for d in my_data_timed_b:
    note_listb.append([
        d['beat'] - start_timeb,
        mag_to_pitch_tunedb(d['freq']),
        100,  # velocity
        1  # duration, in beats
    ])


In [ ]:
# Add a track with those notes
mymidi.add_track(note_listb)
# Output the .mid file
mymidi.save_midi()

In [ ]:
from pydub import AudioSegment
from pydub.generators import Sine
from collections import defaultdict
from mido import MidiFile

In [ ]:
def note_to_freq(note, concert_A=440.0):
    
    return (2.0 ** ((note - 69) / 12.0)) * concert_A

mid = MidiFile("histilist.mid")
output = AudioSegment.silent(mid.length * 2000.0)

tempo = 150 # bpm
output = output.low_pass_filter(1000)

def ticks_to_ms(ticks):
    tick_ms = (60000.0 / tempo) / mid.ticks_per_beat
    return ticks * tick_ms
  

for track in mid.tracks:
  
    current_pos = 0.0

    current_notes = defaultdict(dict)

    for msg in track:
        current_pos += ticks_to_ms(msg.time)

        if msg.type == 'note_on':
            current_notes[msg.channel][msg.note] = (current_pos, msg)

        if msg.type == 'note_off':
            start_pos, start_msg = current_notes[msg.channel].pop(msg.note)

            duration = current_pos - start_pos

            signal_generator = Sine(note_to_freq(msg.note))
            rendered = signal_generator.to_audio_segment(duration=duration, volume=-20).fade_out(300).fade_in(30)

            output = output.overlay(rendered, start_pos)

output.export("histlist.mp3", format="mp3")

In [ ]:
from pedalboard import *
from pedalboard.io import AudioFile

with AudioFile('histlist.mp3', 'r') as f:
    audio = f.read(f.frames)
    samplerate = f.samplerate

# Make a pretty interesting sounding guitar pedalboard:
board = Pedalboard([
    Compressor(),
    Reverb(room_size=1),
    Delay(delay_seconds=0.24, mix=0.8)
])

# Pedalboard objects behave like lists, so you can add plugins:
board.append(Gain(gain_db=10 ))


# Run the audio through this pedalboard!
effected = board(audio, samplerate)


# Write the audio back as a wav file:
with AudioFile('output.mp3', 'w', samplerate, effected.shape[0]) as f:
    f.write(effected)

song = AudioSegment.from_mp3("output.mp3")
song=song.fade_in(2000).fade_out(3000)
song.export("output.mp3", format="mp3")

In [ ]:
import librosa, librosa.display
from mobilechelonian import Turtle

In [ ]:
name ="output"
file = name+".mp3"
title =name+".ps"
png = name + '.png'
frame_count=0


In [ ]:
signal, sample_rate = librosa.load(file)

In [ ]:
FRAME_SIZE = 2028
HOP_LENGTH = 10000

In [ ]:
rms = librosa.feature.rms(signal, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH)[0]

In [ ]:
spec = []
for i in rms:
    spec.append(i*10000)

In [ ]:
len(spec)

In [ ]:
import turtle
import time
from turtle import Screen, Turtle
from random import randint
import random
import PIL.Image
from tkinter import *
import itertools
import io

In [ ]:
current_directory = os.getcwd()
image_path = os.path.join(current_directory, r'visual')
if not os.path.exists(image_path):
    os.makedirs(image_path)

window = turtle.Screen()
window.title(str(name))
window.setup(width = 650, height = 650)
cv = turtle.getcanvas()
turtle.hideturtle()
turtle.speed(0)



colors = ['white','yellow'
         ]
angle = [95,105,115,125,135,145,155,165,175]
width = [0,2,5]
turtle.penup()

turtle.penup()
turtle.pencolor('white')
turtle.width(3)
turtle.fillcolor('black')
turtle.begin_fill()
turtle.goto(-300,-300)
turtle.pendown()
turtle.goto(300,-300)
turtle.goto(300,300)
turtle.goto(-300,300)
turtle.goto(-300,-300)
turtle.end_fill()
turtle.penup()


for s in spec:
    c= random.choice(colors)
    w=random.choice(width)
    a= random.choice(angle)
    turtle.goto(randint(-50,50),-50)
    turtle.setheading(90)
    turtle.pendown()
    turtle.pencolor(c)
    turtle.width(w)
    turtle.pendown
    turtle.forward(i*7000)
    turtle.penup()
        
        
cv.postscript(file=title, colormode='color')

img = PIL.Image.open(title)
img.load(scale=10)  
img.save(png,'png')  #final result from sound to image


turtle.done()

print('is done')

In [1]:
from moviepy.editor import *
from natsort import natsorted

In [2]:
#compiling image & sound-
videoclip = ImageClip("output.png")
audioclip = AudioFileClip("output.mp3")
videoclip = videoclip.set_duration(audioclip.duration)
videoclip = videoclip.set_audio(AudioFileClip("output.mp3"))
videoclip.write_videofile('output.mp4',fps=4,temp_audiofile='temp-audio.m4a', remove_temp=True, codec="libx264", audio_codec="aac")

Moviepy - Building video output.mp4.
MoviePy - Writing audio in temp-audio.m4a


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
